In [1]:
import pandas as pd
import datetime as dt
import calendar
import string
import contractions
import unicodedata
import sklearn as sk
import sys

In [2]:
sys.path.insert(1,'../src')
import general_tools as gt
import etl

In [3]:
config = pd.read_csv('../bucket/config/config.csv',index_col=0)

* The commented code could be used in future iterations for additional features

In [4]:
punct_to_remove=string.punctuation
raw_data = pd.read_csv(config.loc['iphone6_tweets_repository'][0])
# print(raw_data.loc[1],'\n\n',dt.datetime.strptime(raw_data.loc[1].date,'%b %d, %Y').weekday())
# weekend_vs_weekend=[]
raw_data=raw_data.sort_values(by='date',ascending=False).reset_index(drop=True)
has_url=[] # technically could be part of feature engineering but because text cleanup reshapes url it is done here
txt=[]

#to sort by date, raw_date is three-letter-month day, year format string rather than datetime
raw_data['date']=pd.to_datetime(raw_data['date'])
raw_data=raw_data.sort_values(by='date',ascending=True).reset_index(drop=True)
raw_data['date']=raw_data['date'].apply(lambda _: dt.datetime.strftime(_,'%b %d, %Y'))

# cleaning text for NLP and checking if tweet has url
for i in range(raw_data.shape[0]):
    # weekend_vs_weekend.append('w_day' if dt.datetime.strptime(raw_data.loc[i].date,'%b %d, %Y').weekday()<5 else 'w_end')
    has_url.append(int('http://' in raw_data.loc[i].text or 'https://' in raw_data.loc[i].text))# checks if tweets has a link
    txt.append(unicodedata.normalize('NFKD',contractions.fix(raw_data.loc[i].text.replace('\n',' ').lower().\
            translate(str.maketrans('','',punct_to_remove)))).encode('ascii','ignore').decode('utf-8','ignore'))# text cleanup 
etl_data=raw_data.copy()
etl_data.drop(columns=['date','text'],inplace=True) #current iteration of the model doesn't use date in feature embedding; in future iterations it should because it enhances the count_perID feature
etl_data['text'] = txt
etl_data['has_url']=has_url
# raw_data['Weekday_vs_Weekend']=weekend_vs_weekend

count_values=etl_data['id'].value_counts(sort=False)
etl_data['tweet_count_perID_fromStart'] = [0]*etl_data.shape[0]
for i in range(etl_data.shape[0]-1,-1,-1):
    etl_data.loc[i,'tweet_count_perID_fromStart']= count_values[etl_data.loc[i,'id']]
    count_values[etl_data.loc[i,'id']]-=1


In [4]:
# etl_d=etl.perform_etl()
# etl_d.head(10)

,name,id,text,has_url
0,Not Nullible,@ElonIsTrash_lol,monogram damask design otterbox commuter ipho...,1
1,Minding Mine,@No1DisTracTsMe,wayne said suck a n d for a iphone 6,0
2,iPhone shows,@iPhoneshows,best waterproof iphone 6 cases got to be mobi...,1
3,I'm hi,@mOOnmanZOOTED,fixed up this iphone 6 11,0
4,About iPhone,@AboutiPhone14,how to download taig jailbreak ios813 for ipho...,1
5,iPhone News,@iPhonenew3,how to download taig jailbreak ios813 for ipho...,1
6,Brandon,@ayo_debow,replying to kordelliciouskordellicious suck a ...,0
7,Tech4YourNeeds,@Tech4YourNeeds,i liked a youtube video from premierreviewer h...,1
8,Sabrina,@sabrinasho12,phone 8 iphone 6 plus screen protector cellbee...,1
9,NadirahK.,@NizzyNaa,treating myself to the iphone 6 for my birthda...,0


In [5]:
# full_dataset=etl.feature_addition(etl_d)
# full_dataset.head(10)

,name,id,text,has_url,tweet_count_perID_fromStart
0,Not Nullible,@ElonIsTrash_lol,monogram damask design otterbox commuter ipho...,1,1
1,Minding Mine,@No1DisTracTsMe,wayne said suck a n d for a iphone 6,0,1
2,iPhone shows,@iPhoneshows,best waterproof iphone 6 cases got to be mobi...,1,1
3,I'm hi,@mOOnmanZOOTED,fixed up this iphone 6 11,0,1
4,About iPhone,@AboutiPhone14,how to download taig jailbreak ios813 for ipho...,1,1
5,iPhone News,@iPhonenew3,how to download taig jailbreak ios813 for ipho...,1,1
6,Brandon,@ayo_debow,replying to kordelliciouskordellicious suck a ...,0,1
7,Tech4YourNeeds,@Tech4YourNeeds,i liked a youtube video from premierreviewer h...,1,1
8,Sabrina,@sabrinasho12,phone 8 iphone 6 plus screen protector cellbee...,1,1
9,NadirahK.,@NizzyNaa,treating myself to the iphone 6 for my birthda...,0,1


In [6]:
# etl.save_post_etl_and_featureEng(full_dataset)